## importing the liabraries

In [1]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sai\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sai\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sai\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sai\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Sai\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
from nltk.tokenize import sent_tokenize,word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords
import string
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

## importing Dataset

In [3]:
dataset=pd.read_csv('Complete dataset.csv')[0:4000]

In [4]:
dataset.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [5]:
text=dataset['text'].values
text

array(['@VirginAmerica What @dhepburn said.',
       "@VirginAmerica plus you've added commercials to the experience... tacky.",
       "@VirginAmerica I didn't today... Must mean I need to take another trip!",
       ...,
       '@united Yes, though they have not located the bag yet. It was a lovely flight outside of that.',
       '@united Got me home amid snow &amp; Cancelled Flightlations; delivered bag w/o hassle; plus no wait on phone &amp; an upgrade. Thank you!',
       '@united My wife has been on the phone several times for past 3 days, with no result.'],
      dtype=object)

In [6]:
text=[word_tokenize(doc) for doc in text]
sentiment=dataset['airline_sentiment'].values

In [7]:
documents=[(text[i],sentiment[i]) for i in range (4000)]

In [8]:
documents[0]

(['@', 'VirginAmerica', 'What', '@', 'dhepburn', 'said', '.'], 'neutral')

## Cleaning dataset

In [9]:
lemmatizer=WordNetLemmatizer()

In [10]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [11]:
stops=set(stopwords.words('english'))
punctuations=list(string.punctuation)
stops.update(punctuations)
stops,string.punctuation

({'!',
  '"',
  '#',
  '$',
  '%',
  '&',
  "'",
  '(',
  ')',
  '*',
  '+',
  ',',
  '-',
  '.',
  '/',
  ':',
  ';',
  '<',
  '=',
  '>',
  '?',
  '@',
  '[',
  '\\',
  ']',
  '^',
  '_',
  '`',
  'a',
  'about',
  'above',
  'after',
  'again',
  'against',
  'ain',
  'all',
  'am',
  'an',
  'and',
  'any',
  'are',
  'aren',
  "aren't",
  'as',
  'at',
  'be',
  'because',
  'been',
  'before',
  'being',
  'below',
  'between',
  'both',
  'but',
  'by',
  'can',
  'couldn',
  "couldn't",
  'd',
  'did',
  'didn',
  "didn't",
  'do',
  'does',
  'doesn',
  "doesn't",
  'doing',
  'don',
  "don't",
  'down',
  'during',
  'each',
  'few',
  'for',
  'from',
  'further',
  'had',
  'hadn',
  "hadn't",
  'has',
  'hasn',
  "hasn't",
  'have',
  'haven',
  "haven't",
  'having',
  'he',
  'her',
  'here',
  'hers',
  'herself',
  'him',
  'himself',
  'his',
  'how',
  'i',
  'if',
  'in',
  'into',
  'is',
  'isn',
  "isn't",
  'it',
  "it's",
  'its',
  'itself',
  'just',
  'll',


In [12]:
def clean_review(words):
    output_words=[]
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
            return output_words

In [13]:
document=[(clean_review(doc),category) for doc,category in documents]

In [14]:
for i in document:
    print(i)

(['virginamerica'], 'neutral')
(['virginamerica'], 'positive')
(['virginamerica'], 'neutral')
(['virginamerica'], 'negative')
(['virginamerica'], 'negative')
(['virginamerica'], 'negative')
(['virginamerica'], 'positive')
(['virginamerica'], 'neutral')
(['virginamerica'], 'positive')
(['virginamerica'], 'positive')
(['virginamerica'], 'neutral')
(['virginamerica'], 'positive')
(['virginamerica'], 'positive')
(['virginamerica'], 'positive')
(['virginamerica'], 'positive')
(['virginamerica'], 'negative')
(['virginamerica'], 'positive')
(['virginamerica'], 'negative')
(['❤️'], 'positive')
(['virginamerica'], 'positive')
(['virginamerica'], 'negative')
(['virginamerica'], 'positive')
(['virginamerica'], 'positive')
(['virginamerica'], 'neutral')
(['virginamerica'], 'negative')
(['virginamerica'], 'negative')
(['virginamerica'], 'negative')
(['virginamerica'], 'neutral')
(['virginamerica'], 'negative')
(['virginamerica'], 'neutral')
(['virginamerica'], 'negative')
(['virginamerica'], 'neutr

In [15]:
categories=[category for document,category in documents]
categories

['neutral',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'positive',
 'positive',
 'positive',
 'positive',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'positive',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'positive',
 'positive',
 'neutral',
 'negative',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'neutral',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'ne

In [16]:
text_documents=[' '.join(document) for document,category in documents]
text_documents

['@ VirginAmerica What @ dhepburn said .',
 "@ VirginAmerica plus you 've added commercials to the experience ... tacky .",
 "@ VirginAmerica I did n't today ... Must mean I need to take another trip !",
 "@ VirginAmerica it 's really aggressive to blast obnoxious `` entertainment '' in your guests ' faces & amp ; they have little recourse",
 "@ VirginAmerica and it 's a really big bad thing about it",
 "@ VirginAmerica seriously would pay $ 30 a flight for seats that did n't have this playing . it 's really the only bad thing about flying VA",
 '@ VirginAmerica yes , nearly every time I fly VX this “ ear worm ” won ’ t go away : )',
 '@ VirginAmerica Really missed a prime opportunity for Men Without Hats parody , there . https : //t.co/mWpG7grEZP',
 "@ virginamerica Well , I didn't…but NOW I DO ! : -D",
 "@ VirginAmerica it was amazing , and arrived an hour early . You 're too good to me .",
 '@ VirginAmerica did you know that suicide is the second leading cause of death among teens 1

## splitting the dataset in training and testing set

In [17]:
x_train,x_test,y_train,y_test=train_test_split(text_documents,categories)

## vectorization to get max frequency words

In [18]:
count_vect=TfidfVectorizer(max_features=5000,max_df=0.8,min_df=0.001)
x_train_features=count_vect.fit_transform(x_train)
x_test_features=count_vect.transform(x_test)

In [19]:
x_train_features

<3000x1713 sparse matrix of type '<class 'numpy.float64'>'
	with 40962 stored elements in Compressed Sparse Row format>

In [20]:
x_test_features

<1000x1713 sparse matrix of type '<class 'numpy.float64'>'
	with 13000 stored elements in Compressed Sparse Row format>

## using SVM classifier

In [21]:
clf=SVC()
clf.fit(x_train_features,y_train)
ypred1=clf.predict(x_test_features)
clf.score(x_test_features,y_test)

0.732

In [22]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,ypred1,labels=['negative','positive'])

array([[621,   4],
       [ 76,  67]], dtype=int64)